Reference: https://towardsdatascience.com/a-practitioners-guide-to-natural-language-processing-part-i-processing-understanding-text-9f4abfd13e72 <br>AND Annie's "Text Normalization Demo.ipynb"

# Data Exploratory

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline

In [2]:
df = pd.read_csv('training.csv')

In [3]:
type(df)

pandas.core.frame.DataFrame

In [4]:
len(df)

32000

In [5]:
df.count() # Check number of non-NA values

Tweet        32000
Sentiment    32000
dtype: int64

In [6]:
df.head(5) # "4" represents "positive", "0" represents "negative"

,Tweet,Sentiment
0,"havin relaxin nite, drinkin earl grey &amp; wa...",4
1,@atif089 cool that would be nice,4
2,... i HATE lyn-z... sorry i just had to say it...,0
3,"is awake, bored, and annoyed",0
4,song of the day http://tinyurl.com/cpkjrm lac...,4


In [7]:
df.apply(pd.value_counts).head() # Check and count all duplicated tweets

c:\users\tblakeley\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexes\api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


,Tweet,Sentiment
isPlayer Has Died! Sorry,6.0,NaN
Good morning,5.0,NaN
So bored,3.0,NaN
My friends made me join twitter.. so here i am hows everyones day so far?,3.0,NaN
Good Morning,3.0,NaN


In [8]:
df.count() # Reduce tweets from 32000 to 31944

Tweet        32000
Sentiment    32000
dtype: int64

In [9]:
training_df = df
training_df.count() # Reduce tweets from 31944 to 31938

Tweet        32000
Sentiment    32000
dtype: int64

# Data Cleaning - Import necessary dependencies

In [10]:
import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
from contractions import CONTRACTION_MAP
import unicodedata

# nltk.download('stopwords')
# python -m spacy download en

nlp = spacy.load('en', parse=True, tag=True, entity=True)
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

c:\users\tblakeley\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
c:\users\tblakeley\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)
c:\users\tblakeley\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
c:\users\tblakeley\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds

In [11]:
training_df1=training_df #Create training_df['Tweet']1 as a back-up

# Lowercase the text 

In [12]:
training_df['Tweet']=training_df['Tweet'].str.lower()

In [13]:
training_df['Tweet'].head(10)

0    havin relaxin nite, drinkin earl grey &amp; wa...
1                    @atif089 cool that would be nice 
2    ... i hate lyn-z... sorry i just had to say it...
3                        is awake, bored, and annoyed 
4    song of the day  http://tinyurl.com/cpkjrm lac...
5    @prettyhoneydip iight lol ; i got u when i tal...
6            thinks she has run up a â£300 phone bill 
7    @jessicagottlieb  prime example why it doesn't...
8                        @bettie_mcfly you know, why? 
9    @mcrmuffin d i got your update on my phone!! y...
Name: Tweet, dtype: object

# Remove @Word and "#Word"

In [14]:
def removeTags(s):
    s = re.sub("@[a-zA-Z0-9_]+"," ", s) #remove the @<the word>
    #s = re.sub("#[a-zA-Z0-9_]+"," ", s) #remove the #<the word>
    
    return s

training_df['Tweet']=training_df['Tweet'].apply(removeTags)
training_df['Tweet'] = training_df['Tweet'].str.replace('#','')

In [15]:
training_df['Tweet'].head(10)

0    havin relaxin nite, drinkin earl grey &amp; wa...
1                             cool that would be nice 
2    ... i hate lyn-z... sorry i just had to say it...
3                        is awake, bored, and annoyed 
4    song of the day  http://tinyurl.com/cpkjrm lac...
5      iight lol ; i got u when i talk 2 him. im no...
6            thinks she has run up a â£300 phone bill 
7       prime example why it doesn't really pay to ...
8                                      you know, why? 
9      d i got your update on my phone!! yus! wats ...
Name: Tweet, dtype: object

# Removing HTML and HTTP Tags

In [16]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

training_df['Tweet']=training_df['Tweet'].apply(strip_html_tags)

In [17]:
training_df['Tweet'].head(10)

0    havin relaxin nite, drinkin earl grey & watchi...
1                             cool that would be nice 
2    ... i hate lyn-z... sorry i just had to say it...
3                        is awake, bored, and annoyed 
4    song of the day  http://tinyurl.com/cpkjrm lac...
5      iight lol ; i got u when i talk 2 him. im no...
6            thinks she has run up a â£300 phone bill 
7       prime example why it doesn't really pay to ...
8                                      you know, why? 
9      d i got your update on my phone!! yus! wats ...
Name: Tweet, dtype: object

In [18]:
def removewebsite(http):
    http = re.sub(r"http\S+", "", http) #remove the http://<the word>
    www =  re.sub(r'www.[^ ]+', "", http)
    return www

training_df['Tweet']=training_df['Tweet'].apply(removewebsite)

In [19]:
training_df['Tweet'].head(10)

0    havin relaxin nite, drinkin earl grey & watchi...
1                             cool that would be nice 
2    ... i hate lyn-z... sorry i just had to say it...
3                        is awake, bored, and annoyed 
4                     song of the day   laceys awesome
5      iight lol ; i got u when i talk 2 him. im no...
6            thinks she has run up a â£300 phone bill 
7       prime example why it doesn't really pay to ...
8                                      you know, why? 
9      d i got your update on my phone!! yus! wats ...
Name: Tweet, dtype: object

# Removing accented characters

In [20]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

training_df['Tweet']=training_df['Tweet'].apply(remove_accented_chars)

In [21]:
training_df['Tweet'].head(10)

0    havin relaxin nite, drinkin earl grey & watchi...
1                             cool that would be nice 
2    ... i hate lyn-z... sorry i just had to say it...
3                        is awake, bored, and annoyed 
4                     song of the day   laceys awesome
5      iight lol ; i got u when i talk 2 him. im no...
6             thinks she has run up a a300 phone bill 
7       prime example why it doesn't really pay to ...
8                                      you know, why? 
9      d i got your update on my phone!! yus! wats ...
Name: Tweet, dtype: object

# Expanding Contractions

In [22]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

training_df['Tweet']=training_df['Tweet'].apply(expand_contractions)

In [23]:
training_df['Tweet'].head(10)
training_df["Tweet"][1160]

'i am soooo board '

# Remove Single Letters

In [24]:
def remove_single_letters(text):
    #re.sub("[^a-zA-Z]", " ", neg_handled)
    letters_only = re.sub("[^a-zA-Z]", " ", text)
    words = [x for x  in tokenizer.tokenize(letters_only) if len(x) > 1]
    
    return (" ".join(words)).strip()

training_df['Tweet'] = training_df['Tweet'].apply(remove_single_letters)

# Insert spaces between special characters to isolate them 

In [25]:
def insertspace(inspace):
    special_char_pattern = re.compile(r'([{.(-)!}])')
    inspace = special_char_pattern.sub(" \\1 ", inspace)
    return inspace

training_df['Tweet']=training_df['Tweet'].apply(insertspace)

# Remove numbers

In [26]:
def removeNumbers(n):
    n = re.sub("1[a-zA-Z0-9_]+"," ", n) 
    n = re.sub("2[a-zA-Z0-9_]+"," ", n) 
    n = re.sub("3[a-zA-Z0-9_]+"," ", n) 
    n = re.sub("4[a-zA-Z0-9_]+"," ", n) 
    n = re.sub("5[a-zA-Z0-9_]+"," ", n) 
    n = re.sub("6[a-zA-Z0-9_]+"," ", n) 
    n = re.sub("7[a-zA-Z0-9_]+"," ", n) 
    n = re.sub("8[a-zA-Z0-9_]+"," ", n) 
    n = re.sub("9[a-zA-Z0-9_]+"," ", n) 
    n = re.sub("[0-9]"," ", n) 
    return n

training_df['Tweet']=training_df['Tweet'].apply(removeNumbers)

# Removing Special Characters

In [27]:
def remove_special_characters(text):
    text = re.sub('[^a-zA-Z0-9\s+]', '', text)
    text = re.sub('[+]', '', text)
    return text

training_df['Tweet']=training_df['Tweet'].apply(remove_special_characters)

In [28]:
training_df['Tweet'].head(10)

0    havin relaxin nite drinkin earl grey watchin v...
1                              cool that would be nice
2                    hate lyn sorry just had to say it
3                           is awake bored and annoyed
4                       song of the day laceys awesome
5              iight lol got when talk him im not home
6                     thinks she has run up phone bill
7    prime example why it does not really pay to au...
8                                         you know why
9      got your update on my phone yus wats matter tho
Name: Tweet, dtype: object

# Lemmatizing text

In [29]:
def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

training_df['Tweet']=training_df['Tweet'].apply(lemmatize_text)

In [30]:
training_df['Tweet'].head(10)

0    have relaxin nite drinkin earl grey watchin ve...
1                              cool that would be nice
2                   hate lyn sorry just have to say it
3                           be awake bored and annoyed
4                        song of the day lacey awesome
5            iight lol get when talk him i be not home
6                     think she have run up phone bill
7    prime example why it do not really pay to auto...
8                                         you know why
9       get your update on my phone yus wat matter tho
Name: Tweet, dtype: object

# Removing Stopwords

In [31]:
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

training_df['Tweet']=training_df['Tweet'].apply(remove_stopwords)

In [32]:
training_df['Tweet'].head(10)

0    relaxin nite drinkin earl grey watchin cool sh...
1                                      cool would nice
2                                   hate lyn sorry say
3                                  awake bored annoyed
4                               song day lacey awesome
5                          iight lol get talk not home
6                                 think run phone bill
7    prime example not really pay auto follow irres...
8                                                 know
9                  get update phone yus wat matter tho
Name: Tweet, dtype: object

# Remove extra newlines

In [33]:
def removeline(line):
    line = re.sub(r'[\r|\n|\r\n]+', ' ',line)
    return line

training_df['Tweet']=training_df['Tweet'].apply(removeline)

# Remove extra whitespace

In [34]:
def removewhite(line):
    line = re.sub(' +', ' ', line)
    return line

training_df['Tweet']=training_df['Tweet'].apply(removewhite)

# Stemming Text

In [35]:
#from nltk.stem import PorterStemmer
#ps = PorterStemmer()
#for word in nlp(training_df['Tweet'][1]):
    #print(word)
    
#def stem_text(text):
    #tokens = tokenizer.tokenize(text)
 
    #stem_tokens = [ps.stem(token) for token in tokens]
    #filtered_text = ' '.join(stem_tokens)    
    #return filtered_text


#training_df['Tweet']=training_df['Tweet'].apply(stem_text)

In [36]:
#training_df['Tweet'].head(10)

In [37]:
training_df.isnull().any()

Tweet        False
Sentiment    False
dtype: bool

In [38]:
training_df.to_csv('training_cleaned.csv',encoding='utf-8',index=False)